## Preprocessing. Read data file 'charity_data.csv', and create dataframe 'application_df'

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head(5)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Output characteristics and properties of dataframe.

In [2]:
application_df.shape

(34299, 12)

In [3]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


### The Dataframe has 12 Columns. They comprise the Target for th emodel "IS_SUCCESSFUL", while the other 11variables are potential Features for the model.

In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)

# Confirm fields have been dropped.
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


### Confirmed that the fields 'EIN' and 'NAME' have been dropped. Now 9 Variables as lists are Features for the model.

## Determine the number of Unique values in each column.

In [5]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


## ## Determine the number of data points for each unique value for columns with more that 10 unique values, in this case APPLICATION_TYPE.

In [6]:
# Look at APPLICATION_TYPE value counts for binning
# Get value counts for the 'APPLICATION_TYPE' column
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Print the value counts
# print(application_type_counts)

## Selected 250 as the cut off point for APPLICATION_TYPES. All counts below 250 are binned as 'other'

In [7]:
# Get value counts for the 'APPLICATION_TYPE' column
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Choose a cutoff value to determine which types to replace
cutoff = 500

# Create a list of application types to be replaced (those below the cutoff)
application_types_to_replace = application_type_counts[application_type_counts < cutoff].index.tolist()

# Print the list of application types to replace
# print(application_types_to_replace)

In [8]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
# application_df['APPLICATION_TYPE'].value_counts()

### Binning is successful

## Selected 1000 as the cut off point for CLASSIFICATION. First counted the numbers of each classification, then removed those with a classification, then binned those below 1000 as 'other'. 

In [9]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Print the value counts
# print(classification_counts)

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classifications_greater_than_one = classification_counts[classification_counts > 1]

# Print the filtered classifications and their counts
# print(classifications_greater_than_one)

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
cutoff = 1000

# Create a list of classifications to be replaced (those below the cutoff)
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
# application_df['CLASSIFICATION'].value_counts()

### Encode Categorical variables using pd.get_dummies()

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_numeric = pd.get_dummies(application_df)

# Print the first first of the new DataFrame to verify the transformation
# print(application_df_numeric.head(5))

## Split this preproccessed data into a Target array (IS_SUCCESSFUL), and Features array.

In [13]:
# Extract the target variable into its own array
y = application_df_numeric['IS_SUCCESSFUL'].values

# Drop the target column from the DataFrame and use the rest as the features array
X = application_df_numeric.drop(['IS_SUCCESSFUL'], axis=1).values

# Print the shapes of X and y to confirm the split
print("Shape of Features Matrix:", X.shape)
print("Shape of Target Array:", y.shape)

Shape of Features Matrix: (34299, 43)
Shape of Target Array: (34299,)


### Split the data set into train and test

In [14]:
# Split the data set into train and test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the train/test datasets to verify the split
print("Training set shape (features):", X_train.shape)
print("Test set shape (features):", X_test.shape)
print("Training set shape (target):", y_train.shape)
print("Test set shape (target):", y_test.shape)
#  YOUR CODE GOES HERE

Training set shape (features): (27439, 43)
Test set shape (features): (6860, 43)
Training set shape (target): (27439,)
Test set shape (target): (6860,)


## Create a StandardScaler instances

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Fit the Two Hidden Layer Model, activation Function Tanh, Tanh over 50 Epochs.

In [16]:
#### Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
n_features = X_train_scaled.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(32, input_dim=n_features, activation='tanh'))

# Second hidden layer
nn.add(Dense(16, activation='tanh')) 

# Output layer
nn.add(Dense(1, activation='sigmoid'))


C:\Users\jerry\anaconda1\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
# Model summary
nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,953 (7.63 KB)

 Trainable params: 1,953 (7.63 KB)

 Non-trainable params: 0 (0.00 B)

### Confirm the data type for X_train, and y_train.

In [19]:
print("Shape of X_train_scaled:", X_train_scaled.shape)
print("Shape of y_train_:", y_train.shape)

# Convert X_train and y_train to float32 if they are not already
X_train_scaled = X_train_scaled.astype('float32')
y_train = y_train.astype('float32')

Shape of X_train_scaled: (27439, 43)
Shape of y_train_: (27439,)


### Train the Model

In [20]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=50, batch_size=10, validation_split=0.2, verbose=1)

Epoch 1/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - accuracy: 0.6979 - loss: 0.5900 - val_accuracy: 0.7380 - val_loss: 0.5504
Epoch 2/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.7271 - loss: 0.5529 - val_accuracy: 0.7385 - val_loss: 0.5495
Epoch 3/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.7326 - loss: 0.5505 - val_accuracy: 0.7352 - val_loss: 0.5488
Epoch 4/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.7274 - loss: 0.5525 - val_accuracy: 0.7329 - val_loss: 0.5504
Epoch 5/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.7274 - loss: 0.5520 - val_accuracy: 0.7358 - val_loss: 0.5478
Epoch 6/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.7353 - loss: 0.5439 - val_accuracy: 0.7371 - val_loss: 0.5465
Epoch 7/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 22s 10ms/step - accuracy: 0.7255 - loss: 0.5527 - val_accuracy: 0.7382 - val_loss: 0.5447
Epoch 8/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.7302 - loss: 

2196/2196 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.7369 - loss: 0.5372 - val_accuracy: 0.7391 - val_loss: 0.5444


### Output 2 Hidden Layer with Tanh Activation Functions and 50 Epochs

In [21]:
# Print out the final metrics
history_dict = history.history

# Retrieve the final metrics from the last epoch
final_train_loss = history_dict['loss'][-1]
final_train_accuracy = history_dict['accuracy'][-1]
final_validation_loss = history_dict['val_loss'][-1]
final_validation_accuracy = history_dict['val_accuracy'][-1]

# Print out the final metrics
print(f"Final training loss: {final_train_loss:.4f}")
print(f"Final training accuracy: {final_train_accuracy:.4f}")
print(f"Final validation loss: {final_validation_loss:.4f}")
print(f"Final validation accuracy: {final_validation_accuracy:.4f}")

Final training loss: 0.5371
Final training accuracy: 0.7377
Final validation loss: 0.5444
Final validation accuracy: 0.7391


### Fit the Four Hidden Layer Model, activation Function Tanh, Tanh, Relu, Relu over 100 Epochs.

In [22]:
#### Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
n_features = X_train_scaled.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(128, input_dim=n_features, activation='tanh'))

# Second hidden layer
nn.add(Dense(64, activation='tanh')) 

# Third hidden layer
nn.add(Dense(32, activation = 'relu'))

# fourth hidden layer
nn.add(Dense(16, activation = 'relu'))

# Output layer
nn.add(Dense(1, activation='sigmoid'))
# Compile the model

nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
nn.summary()

### Confirm the data type for X_train, and y_train.

print("Shape of X_train_scaled:", X_train_scaled.shape)
print("Shape of y_train_:", y_train.shape)

# Convert X_train and y_train to float32 if they are not already
X_train_scaled = X_train_scaled.astype('float32')
y_train = y_train.astype('float32')

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │         5,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,513 (64.50 KB)

 Trainable params: 16,513 (64.50 KB)

 Non-trainable params: 0 (0.00 B)

Shape of X_train_scaled: (27439, 43)
Shape of y_train_: (27439,)


In [23]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=10, validation_split=0.2, verbose=1)

Epoch 1/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 32s 9ms/step - accuracy: 0.7156 - loss: 0.5778 - val_accuracy: 0.7371 - val_loss: 0.5565
Epoch 2/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 25s 11ms/step - accuracy: 0.7244 - loss: 0.5564 - val_accuracy: 0.7382 - val_loss: 0.5540
Epoch 3/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 27s 12ms/step - accuracy: 0.7311 - loss: 0.5522 - val_accuracy: 0.7387 - val_loss: 0.5465
Epoch 4/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 28s 13ms/step - accuracy: 0.7305 - loss: 0.5496 - val_accuracy: 0.7387 - val_loss: 0.5469
Epoch 5/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 28s 12ms/step - accuracy: 0.7289 - loss: 0.5510 - val_accuracy: 0.7396 - val_loss: 0.5513
Epoch 6/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.7304 - loss: 0.5472 - val_accuracy: 0.7378 - val_loss: 0.5449
Epoch 7/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.7272 - loss: 0.5532 - val_accuracy: 0.7392 - val_loss: 0.5456
Epoch 8/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.73

2196/2196 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.7416 - loss: 0.5340 - val_accuracy: 0.7392 - val_loss: 0.5539
Epoch 51/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.7375 - loss: 0.5338 - val_accuracy: 0.7403 - val_loss: 0.5540
Epoch 52/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.7419 - loss: 0.5276 - val_accuracy: 0.7391 - val_loss: 0.5566
Epoch 53/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7432 - loss: 0.5280 - val_accuracy: 0.7405 - val_loss: 0.5565
Epoch 54/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.7391 - loss: 0.5337 - val_accuracy: 0.7387 - val_loss: 0.5678
Epoch 55/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.7398 - loss: 0.5328 - val_accuracy: 0.7396 - val_loss: 0.5557
Epoch 56/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7365 - loss: 0.5361 - val_accuracy: 0.7380 - val_loss: 0.5575
Epoch 57/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7403 - loss

2196/2196 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7355 - loss: 0.5348 - val_accuracy: 0.7392 - val_loss: 0.5673
Epoch 100/100
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7418 - loss: 0.5303 - val_accuracy: 0.7385 - val_loss: 0.5649


### Output of 4 Hidden Layer Model, Tanh, Tanh, Relu, Relu Activation Function and 100 Epochs

In [24]:
# Print out the final metrics
history_dict = history.history

# Retrieve the final metrics from the last epoch
final_train_loss = history_dict['loss'][-1]
final_train_accuracy = history_dict['accuracy'][-1]
final_validation_loss = history_dict['val_loss'][-1]
final_validation_accuracy = history_dict['val_accuracy'][-1]

# Print out the final metrics
print(f"Final training loss: {final_train_loss:.4f}")
print(f"Final training accuracy: {final_train_accuracy:.4f}")
print(f"Final validation loss: {final_validation_loss:.4f}")
print(f"Final validation accuracy: {final_validation_accuracy:.4f}")

Final training loss: 0.5320
Final training accuracy: 0.7399
Final validation loss: 0.5649
Final validation accuracy: 0.7385


### Fit the 1 Hidden Layer with Tanh Activation Function and 100 Epochs

In [25]:
#### Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
n_features = X_train_scaled.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(64, input_dim=n_features, activation='relu'))

# Second hidden layer
#nn.add(Dense(64, activation='tanh')) 

# Third hidden layer
#nn.add(Dense(32, activation = 'tanh'))

# fourth hidden layer
#nn.add(Dense(16, activation = 'tanh'))

# Output layer
nn.add(Dense(1, activation='sigmoid'))
# Compile the model

nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
nn.summary()

### Confirm the data type for X_train, and y_train.

print("Shape of X_train_scaled:", X_train_scaled.shape)
print("Shape of y_train_:", y_train.shape)

# Convert X_train and y_train to float32 if they are not already
X_train_scaled = X_train_scaled.astype('float32')
y_train = y_train.astype('float32')


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 64)             │         2,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

Shape of X_train_scaled: (27439, 43)
Shape of y_train_: (27439,)


In [26]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=10, batch_size=10, validation_split=0.2, verbose=1)


Epoch 1/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.6999 - loss: 0.5915 - val_accuracy: 0.7351 - val_loss: 0.5515
Epoch 2/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7273 - loss: 0.5608 - val_accuracy: 0.7387 - val_loss: 0.5527
Epoch 3/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7260 - loss: 0.5562 - val_accuracy: 0.7389 - val_loss: 0.5507
Epoch 4/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7268 - loss: 0.5571 - val_accuracy: 0.7371 - val_loss: 0.5503
Epoch 5/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7287 - loss: 0.5509 - val_accuracy: 0.7332 - val_loss: 0.5504
Epoch 6/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7318 - loss: 0.5516 - val_accuracy: 0.7391 - val_loss: 0.5481
Epoch 7/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7273 - loss: 0.5541 - val_accuracy: 0.7367 - val_loss: 0.5490
Epoch 8/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7356 - loss: 0.5474 -

### Output of 1 Hidden Layer Model, Relu Activation Function and 100 Epochs

In [27]:
# Print out the final metrics
history_dict = history.history

# Retrieve the final metrics from the last epoch
final_train_loss = history_dict['loss'][-1]
final_train_accuracy = history_dict['accuracy'][-1]
final_validation_loss = history_dict['val_loss'][-1]
final_validation_accuracy = history_dict['val_accuracy'][-1]

# Print out the final metrics
print(f"Final training loss: {final_train_loss:.4f}")
print(f"Final training accuracy: {final_train_accuracy:.4f}")
print(f"Final validation loss: {final_validation_loss:.4f}")
print(f"Final validation accuracy: {final_validation_accuracy:.4f}")


Final training loss: 0.5506
Final training accuracy: 0.7288
Final validation loss: 0.5517
Final validation accuracy: 0.7362


In [28]:
### Fit the 2 Hidden Layers with Relu Activation Function and 100 Epochs

In [29]:
#### Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
n_features = X_train_scaled.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(16, input_dim=n_features, activation='relu'))

# Second hidden layer
nn.add(Dense(32, activation='tanh')) 

# Third hidden layer
#nn.add(Dense(32, activation = 'tanh'))

# fourth hidden layer
#nn.add(Dense(16, activation = 'tanh'))

# Output layer
nn.add(Dense(1, activation='sigmoid'))
# Compile the model

nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
nn.summary()

### Confirm the data type for X_train, and y_train.

print("Shape of X_train_scaled:", X_train_scaled.shape)
print("Shape of y_train_:", y_train.shape)

# Convert X_train and y_train to float32 if they are not already
X_train_scaled = X_train_scaled.astype('float32')
y_train = y_train.astype('float32')


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 16)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,281 (5.00 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

Shape of X_train_scaled: (27439, 43)
Shape of y_train_: (27439,)


In [30]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=20, batch_size=10, validation_split=0.2, verbose=1)

Epoch 1/20
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.6959 - loss: 0.5982 - val_accuracy: 0.7343 - val_loss: 0.5582
Epoch 2/20
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7211 - loss: 0.5665 - val_accuracy: 0.7310 - val_loss: 0.5530
Epoch 3/20
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7349 - loss: 0.5493 - val_accuracy: 0.7400 - val_loss: 0.5477
Epoch 4/20
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7279 - loss: 0.5562 - val_accuracy: 0.7343 - val_loss: 0.5511
Epoch 5/20
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7279 - loss: 0.5548 - val_accuracy: 0.7385 - val_loss: 0.5478
Epoch 6/20
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7322 - loss: 0.5516 - val_accuracy: 0.7380 - val_loss: 0.5443
Epoch 7/20
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7276 - loss: 0.5486 - val_accuracy: 0.7365 - val_loss: 0.5466
Epoch 8/20
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7305 - loss: 0.5475 

### Output of 2 Hidden Layer Model, Relu Activation Function and 100 Epochs, and Lo Neurons

In [31]:
# Print out the final metrics
history_dict = history.history

# Retrieve the final metrics from the last epoch
final_train_loss = history_dict['loss'][-1]
final_train_accuracy = history_dict['accuracy'][-1]
final_validation_loss = history_dict['val_loss'][-1]
final_validation_accuracy = history_dict['val_accuracy'][-1]

# Print out the final metrics
print(f"Final training loss: {final_train_loss:.4f}")
print(f"Final training accuracy: {final_train_accuracy:.4f}")
print(f"Final validation loss: {final_validation_loss:.4f}")
print(f"Final validation accuracy: {final_validation_accuracy:.4f}")

Final training loss: 0.5453
Final training accuracy: 0.7327
Final validation loss: 0.5437
Final validation accuracy: 0.7383


## Export the Model to an HDFS File called 'AlphabetSoup-Optimising.Charity.h5'

In [39]:
# Export our model to HDF5 file
from tensorflow.keras.models import load_model

# Load the model from the file
nn.save('AlphabetSoup_Optimising.Charity.h5')

# There seems to be aproblem with this cell today (23/04/2024). I was working Yeaster and before.

ImportError: `save_model()` using h5 format requires h5py. Could not import h5py.